In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./daily_csv.csv')
df = df.dropna()
prices = df['Price'].values
minn, maxx = prices.min(), prices.max()
prices = (prices - minn) / (maxx - minn)
X, y = [], []
sequence_length = 10
for i in range(5900):
    l = []
    for j in range(i, i + sequence_length):
        l.append(prices[j])
    X.append(l)
    y.append(prices[i + sequence_length])
X, y = np.array(X), np.array(y)


In [3]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size= 0.10, random_state= 42, stratify= None)

In [4]:
class Data(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x, dtype= torch.float32)
        self.y = torch.tensor(y, dtype= torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [5]:
train = Data(xtrain, ytrain)
test = Data(xtest, ytest)
train_loader = torch.utils.data.DataLoader(train, batch_size= 256, shuffle= True, drop_last= True)
test_loader = torch.utils.data.DataLoader(test, batch_size= 256, drop_last= True)

In [6]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size= 1, hidden_size= 5, batch_first= True, num_layers= 1)
        self.fc1 = torch.nn.Linear(in_features= 5, out_features= 1)
    def forward(self, x):
        y, h = self.lstm(x)
        y = y[:, -1, :]
        return self.fc1(torch.relu(y))

In [7]:
model = Network()
model = model.to('cuda')
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001, weight_decay= 0.0001)

In [8]:
model.train()
for epoch in range(1500):
    running_loss = 0.0
    for input, target in train_loader:
        optimizer.zero_grad()
        input, target = input.reshape((-1, sequence_length, 1)).to('cuda'), target.to('cuda')
        output = model(input).reshape(-1)
        loss = criterion(output, target)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    if epoch % 50 == 0:
        print(f'epoch - {epoch}, loss - {running_loss}')


epoch - 0, loss - 2.630661256611347
epoch - 50, loss - 0.010233054083073512
epoch - 100, loss - 0.00796049420023337
epoch - 150, loss - 0.007129891979275271
epoch - 200, loss - 0.006717320633470081
epoch - 250, loss - 0.006158123258501291
epoch - 300, loss - 0.006205757803400047
epoch - 350, loss - 0.006160298580653034
epoch - 400, loss - 0.00603228453110205
epoch - 450, loss - 0.005967340664938092
epoch - 500, loss - 0.006108430010499433
epoch - 550, loss - 0.00597940749139525
epoch - 600, loss - 0.006088032110710628
epoch - 650, loss - 0.005921397576457821
epoch - 700, loss - 0.006085045810323209
epoch - 750, loss - 0.0059607843577396125
epoch - 800, loss - 0.005957723325991537
epoch - 850, loss - 0.006008519136230461
epoch - 900, loss - 0.0058382974384585395
epoch - 950, loss - 0.006000779067107942
epoch - 1000, loss - 0.006037230021320283
epoch - 1050, loss - 0.005832437047502026
epoch - 1100, loss - 0.005881681441678666
epoch - 1150, loss - 0.006057295388018247
epoch - 1200, loss 

In [9]:
model.eval()
all_pred, all_label = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.reshape((-1, sequence_length, 1)).to('cuda'), target.to('cuda')
        output = model(input).reshape(-1)
        all_pred.extend(output.to('cpu').detach().numpy())
        all_label.extend(target.to('cpu').detach().numpy())

In [10]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(all_pred, all_label))

0.00017887757890132044
